# Create and Insert Data to SQLite

In [35]:
# read tiff file
from spectral_tiffs import read_stiff, read_mtiff
import matplotlib.pyplot as plt
import plotly.express as px


# read tiff and mask tiff file 
tiff_path = "/Users/yusukemikami/Repos/IDP_2022_Project/data/Set 1, lower 10, icg.tiff"
spim, wavelengths, rgb, metadata = read_stiff(tiff_path)

mask_path = "/Users/yusukemikami/Repos/IDP_2022_Project/data/Set 1, lower 10, icg, mask.tiff"
masks = read_mtiff(mask_path)

In [36]:
## save data to sqlite
import sqlite3

conn = sqlite3.connect('hsi_database', detect_types=sqlite3.PARSE_DECLTYPES)
cursor = conn.cursor()

In [37]:
sql_create_table = """ CREATE TABLE IF NOT EXISTS tiff_table (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    spim blob,
                                    rgb blob
                                );"""
count = cursor.execute(sql_create_table)

In [38]:
KEY_TIFF = "Stroma, ICG"
print("spim : ",spim.shape, "wavelengths : ",wavelengths.shape, "rgb : ",rgb.shape, "metadata : ",metadata, "masks : ",masks[KEY_TIFF].shape)

spim :  (1024, 1024, 37) wavelengths :  (37,) rgb :  (1024, 1024, 3) metadata :   masks :  (1024, 1024)


In [55]:
query = u'''insert into tiff_table VALUES(?,?,?,?)'''

cursor.execute(query,(1234,"icgtiff",sqlite3.Binary(spim),sqlite3.Binary(spim)))
conn.commit()

ValueError: could not convert BLOB to buffer

In [ ]:

sqlite_insert_query = f"""INSERT INTO tiff_table
                        (id, name, spim, rgb) 
                        VALUES 
                        (1234,"icgtiff",{sqlite3.Binary(spim)},"")"""

count = cursor.execute(sqlite_insert_query)


OperationalError: near "<": syntax error

In [ ]:
import pandas as pd
df = pd.read_sql_query("SELECT * from tiff_table", conn)
df

,id,name,spim,rgb
